In [1]:
!pip install google-cloud-storage

In [3]:
for bucket in client.list_buckets():
    print(bucket.name)


swipe-bucket


In [5]:
from google.cloud import storage
from PIL import Image
import requests
from io import BytesIO

# 1. Init client
client = storage.Client()
bucket_name = "swipe-bucket"
bucket = client.bucket(bucket_name)

# 2. List all blobs in the bucket
blobs = bucket.list_blobs()

for blob in blobs:
    print(blob.name)  # this prints "01146212.jpg", "02.jpg", etc.


In [7]:
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.8/789.8 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [transformers] [transformers]ub]


In [9]:
!pip install torch torchvision torchaudio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.0/888.0 MB 29.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 38.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 53.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 22.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 55.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 53.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 47.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 54.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [27]:
### Classsification of Images ###

In [28]:
results = []
target_images = 100   # we want 100 non-abstract images
collected = 0

for blob in bucket.list_blobs():
    if collected >= target_images:
        break   # stop when we have 100 valid images

    try:
        img_bytes = blob.download_as_bytes()
        image = Image.open(BytesIO(img_bytes)).convert("RGB")

        # Run CLIP
        inputs = processor(text=labels, images=image, return_tensors="pt", padding=True).to(device)
        outputs = model(**inputs)
        probs = outputs.logits_per_image.softmax(dim=1).cpu().detach().numpy()[0]

        pred_idx = probs.argmax()
        pred_label = labels[pred_idx]

        # ✅ Only keep if not abstract
        if pred_label in ["nature", "city"]:
            results.append([blob.name, pred_label])
            collected += 1

    except Exception as e:
        print(f"Skipping {blob.name} (error: {e})")
        continue

# Save results
df = pd.DataFrame(results, columns=["image_id", "predicted_type"])
df.to_csv("classified_first100_filtered.csv", index=False)

out_blob = bucket.blob("outputs/classified_first100_filtered.csv")
out_blob.upload_from_filename("classified_first100_filtered.csv")

print("✅ Collected exactly 100 non-abstract images and saved to: outputs/classified_first100_filtered.csv")


✅ Collected exactly 100 non-abstract images and saved to: outputs/classified_first100_filtered.csv


In [29]:
# Load back the CSV into a DataFrame
df_filtered = pd.read_csv("classified_first100_filtered.csv")

# Show first 20 rows
print(df_filtered)


                                             image_id predicted_type
0                                        00bcce0f.jpg         nature
1                                        01146212.jpg         nature
2                                        018b51ef.jpg         nature
3                                              02.jpg         nature
4                                        025f1dd8.jpg         nature
5                                              06.jpg         nature
6                                        06b79f51.jpg         nature
7                                              07.jpg         nature
8                                        085bfba0.jpg           city
9                                              09.jpg         nature
10                                       0c412568.jpg         nature
11           0jCul0wRzOXVv9y2bjtp_San_Zenone_WooD.jpg         nature
12                                       12f0a18c.jpg         nature
13                                